### Text Processing for Author Recognition using Spark

#### Import statements

In [8]:
import string

#### Initialized Spark Context

### Read data.json into Spark SQL context

In [1]:
data_file = 'data/data.json'
df = spark.read.json(data_file)

CONSIDER:

for fun use RDD and map reduce to remove the double bars I put into the excerpts!

In [ ]:
print df.printSchema()
print df.count()
df.show(3)

#### NOTE
* Column Objects (like df.excerpt) cannot be called
* Use df.select(col_name).show(num) to view the column

In [ ]:
# print "df.excerpt type = \t\t", type(df.excerpt)
# print "df.select('excerpt') type = \t", type(df.select('excerpt'))
# df.select('excerpt').show(10)

### Here we import some sql funtions and give our dataframe a SQL table name

In [ ]:
from pyspark.sql.functions import length
from pyspark.sql.functions import count

In [ ]:
df.createOrReplaceTempView("excerpts")

In [ ]:
spark.sql(
    "SELECT count(*) FROM excerpts WHERE author = 'MarkTwain'"
    ).show()

SQL queries in Sprak return dataframes. Columns can then be selected through Spark's dataframe syntax

In [ ]:
sqlDF = spark.sql('''
        SELECT excerpt, length(excerpt) AS character_count
        FROM excerpts
        WHERE author = "MarkTwain"
        ''')

sqlDF.show(10)

print "type(sqlDF): \t\t", type(sqlDF)
print "type(sqlDF.character_count): ", type(sqlDF.character_count)

In [ ]:
# individual values seem a little clunky to access . . .

# head returns a list of rows: zeroth row, first col == first character count entry
sqlDF.head(5)[0][1]

# momentary aside to find special chars in books

In [40]:
directory = 'data/books'
weird_chars = {}
count = 0
for book in os.listdir(directory):
    path = os.path.join(directory, book)
    with open(path, 'r') as f:
        text = f.read()
        for c in text:
            if c not in '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ \n\r\t' + string.punctuation:
                weird_chars.add(c)
                count +=1

print count
print weird_chars
    

222121
set(['\x80', '\xc3', '\xe2', '\xc5', '\x84', '\xa6', '\xa9', '\xa0', '\xaa', '\xa3', '\xb1', '\x91', '\xb3', '\xc2', '\xb4', '\xb6', '\x99', '\x98', '\x9d', '\x9c', '\xa2'])


In [35]:
punc = ""
for x in weird_chars:
    punc += x
    
punc
    

'\x80\xc3\xe2\xc5\x84\xa6\xa9\xa0\xaa\xa3\xb1\x91\xb3\xc2\xb4\xb6\x99\x98\x9d\x9c\xa2'

In [36]:
print u'\x80\xc3\xe2\xc5\x84\xa6\xa9\xa0\xaa\xa3\xb1\x91\xb3\xc2\xb4\xb6\x99\x98\x9d\x9c\xa2'

ÃâÅ¦© ª£±³Â´¶¢


In [47]:
print u'\u201c\u201d'
print u'“”'

print u'\xc3'
print u'ÃâÅ¦© ª£±³Â´¶¢'
u'ÃâÅ¦© ª£±³Â´¶¢'

“”
“”
Ã
ÃâÅ¦© ª£±³Â´¶¢


u'\x80\xc3\xe2\xc5\x84\xa6\xa9 \xaa\xa3\xb1\x91\xb3\xc2\xb4\xb6\x99\x98\x9d\x9c\xa2'

In [48]:
'“”'

'\xe2\x80\x9c\xe2\x80\x9d'